# EDA on MSFT historical data
This notebook will use MSFT stock on the EDA.  
We will train the best model identified in the aapl notebook to test how well the model works on other stocks.

### Imports

In [2]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.metrics import f1_score


sys.path.append(str(Path("..").resolve()))
from src.ml_model.data import fetch_data, stock_data_prediction_pipeline, stock_data_feature_engineering, get_one_realtime_bar
from src.ml_model.training import train_model, sequence_split

### Reading Data

In [2]:
df = fetch_data("MSFT", (2020, 1, 1), (2025, 9, 16))
df.head()

open     high       low   close  \
symbol timestamp                                                      
MSFT   2020-01-02 05:00:00+00:00  158.78  160.730  158.3300  160.62   
       2020-01-03 05:00:00+00:00  158.32  159.945  158.0600  158.62   
       2020-01-06 05:00:00+00:00  157.08  159.100  156.5100  159.03   
       2020-01-07 05:00:00+00:00  159.32  159.670  157.3200  157.58   
       2020-01-08 05:00:00+00:00  158.93  160.800  157.9491  160.09   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
MSFT   2020-01-02 05:00:00+00:00  25472631.0     175506.0  159.815510  
       2020-01-03 05:00:00+00:00  24389239.0     166336.0  159.101120  
       2020-01-06 05:00:00+00:00  24709110.0     148395.0  158.495004  
       2020-01-07 05:00:00+00:00  24503429.0     167838.0  158.298227  
       2020-01-08 05:00:00+00:00  31748417.0     198632.0  159.714784

### Preprocessing and Visualization

In [3]:
df_changed = df.copy()
X, y = stock_data_feature_engineering(df_changed)

/home/milad/repositories/Live-Trading-Platform/src/ml_model/data.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = (df['close'].shift(-1) > df['close']).astype(int)


### Training & Evaluation

In [4]:
X_train, X_test, y_train, y_test = sequence_split(X, y)

model = train_model(X_train, y_train)

# Evaluate LSTM
y_pred = (model.predict(X_test) > 0.5).astype(int)
print("LSTM F1-score:", f1_score(y_test, y_pred))

/home/milad/repositories/Live-Trading-Platform/src/ml_model/modelling.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ys.append(y[i + time_steps])
2025-10-12 18:37:17.972943: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
62/62 - 3s - 44ms/step - accuracy: 0.4889 - loss: 0.6985 - val_accuracy: 0.5135 - val_loss: 0.6953
Epoch 2/20
62/62 - 0s - 6ms/step - accuracy: 0.5141 - loss: 0.6936 - val_accuracy: 0.5135 - val_loss: 0.7057
Epoch 3/20
62/62 - 0s - 6ms/step - accuracy: 0.5282 - loss: 0.6900 - val_accuracy: 0.5135 - val_loss: 0.7170
Epoch 4/20
62/62 - 0s - 6ms/step - accuracy: 0.5212 - loss: 0.6891 - val_accuracy: 0.5135 - val_loss: 0.7127
Epoch 5/20
62/62 - 0s - 5ms/step - accuracy: 0.5474 - loss: 0.6876 - val_accuracy: 0.5135 - val_loss: 0.7323
Epoch 6/20
62/62 - 0s - 5ms/step - accuracy: 0.5454 - loss: 0.6906 - val_accuracy: 0.5135 - val_loss: 0.7292
Epoch 7/20
62/62 - 0s - 6ms/step - accuracy: 0.5454 - loss: 0.6863 - val_accuracy: 0.5135 - val_loss: 0.7453
Epoch 8/20
62/62 - 0s - 5ms/step - accuracy: 0.5373 - loss: 0.6859 - val_accuracy: 0.5495 - val_loss: 0.7208
Epoch 9/20
62/62 - 0s - 5ms/step - accuracy: 0.5746 - loss: 0.6809 - val_accuracy: 0.5586 - val_loss: 0.7166
Epoch 10/20
62/62 

In [ ]:
tdf = await get_one_realtime_bar("MSFT")
print(tdf)


In [ ]:
X = stock_data_prediction_pipeline(tdf)

print(X)